# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4958, uptime 0:00:35
mini_beamline                    RUNNING   pid 4959, uptime 0:00:35
random_walk                      RUNNING   pid 4960, uptime 0:00:35
random_walk_horiz                RUNNING   pid 4961, uptime 0:00:35
random_walk_vert                 RUNNING   pid 4962, uptime 0:00:35
simple                           RUNNING   pid 4963, uptime 0:00:35
thermo_sim                       RUNNING   pid 4964, uptime 0:00:35
trigger_with_pc                  FATAL     Exited too quickly (process log may have details)


In [2]:
%run scripts/beamline_configuration.py

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/traitlets/traitlets.py:3036: FutureWarning: --rc={'figure.dpi': 96} for dict-traits is deprecated in traitlets 5.0. You can pass --rc <key=value> ... multiple times to add items to a dict.
  FutureWarning,


/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2021-03-27 22:27:54
Persistent Unique Scan ID: 'c3b68da9-b632-4b00-82b9-468e8830bd43'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 22:27:54.4 |     -1.000 |     104850 |
|         2 | 22:27:54.5 |     -0.500 |     104850 |


|         3 | 22:27:54.5 |      0.000 |     104850 |
|         4 | 22:27:54.5 |      0.500 |     104850 |
|         5 | 22:27:54.5 |      1.000 |     104850 |


+-----------+------------+------------+------------+
generator scan ['c3b68da9'] (scan num: 3)





('c3b68da9-b632-4b00-82b9-468e8830bd43',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/c3b68da9-b632-4b00-82b9-468e8830bd43-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2021-03-27 22:27:54
Persistent Unique Scan ID: '869dbd6c-61ee-40b0-93d3-d00340307dfe'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 22:27:54.9 |          -1 | 1123350222 |


|         2 | 22:27:55.0 |          -0 | 1123350222 |
|         3 | 22:27:55.1 |           0 | 1123350222 |
|         4 | 22:27:55.1 |           0 | 1123350222 |
|         5 | 22:27:55.1 |           1 | 1123350222 |


+-----------+------------+-------------+------------+
generator scan ['869dbd6c'] (scan num: 4)





('869dbd6c-61ee-40b0-93d3-d00340307dfe',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2021-03-27 22:27:55
Persistent Unique Scan ID: '096e44a5-6cad-4bd1-bdb6-9814730e5fc8'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 22:27:55.6 | 1123350222 |


|         2 | 22:27:56.5 | 1123350222 |


|         3 | 22:27:57.5 | 1061698256 |


...

|        28 | 22:28:22.5 | 1064838157 |


|        29 | 22:28:23.5 | 1119488907 |


|        30 | 22:28:24.5 | 1119488907 |


+-----------+------------+------------+
generator count ['096e44a5'] (scan num: 5)





('096e44a5-6cad-4bd1-bdb6-9814730e5fc8',)